In [0]:
from pyspark import *
from pyspark.sql import *
from pyspark.sql.functions import *

In [0]:
dbutils.widgets.text("initial_load_flag", "0")

init_load_flag = int(dbutils.widgets.get("initial_load_flag"))



0


In [0]:
df_customers = spark.read.table("advenworks_cata.silver.customers_silver")
df_customers.display()


In [0]:
df_customers = df_customers.dropDuplicates(subset=['CustomerID'])


**NEW VS OLD RECORDS**

In [0]:
if init_load_flag == 0:

  df_old =  spark.sql(''' select DimCustomerKey, CustomerID, create_date, update_date from advenworks_cata.gold.DimCustomers''')

else:
  df_old =  spark.sql(''' select 0 DimCustomerKey, 0 CustomerID, 0 create_date, 0 update_date from advenworks_cata.silver.customers_silver where 1=0''')

**Renaming Old Records Columns**

In [0]:
df_old = df_old.withColumnRenamed('CustomerID', 'old_CustomerID')\
  .withColumnRenamed('create_date', 'old_create_date')\
  .withColumnRenamed('update_date', 'old_update_date')\
  .withColumnRenamed('DimCustomerKey', 'old_DimCustomerKey')

In [0]:
df_old.display()

**Applying Join With Old Records**

In [0]:
df_joined = df_customers.join(df_old, df_customers.CustomerID == df_old.old_CustomerID, 'left')
df_joined.display()

**Seperating New And Old Records**

In [0]:
df_new = df_joined.filter(df_joined.old_DimCustomerKey.isNull())

df_old = df_joined.filter(df_joined.old_DimCustomerKey.isNotNull())



In [0]:
df_old = df_old.drop('old_CustomerID','old_update_date')

df_old = df_old.withColumnRenamed("old_DimCustomerKey","DimCustomerKey")
df_old = df_old.withColumnRenamed("old_create_date","create_date")

df_old = df_old.withColumn("create_date",to_timestamp(col("create_date")))


df_old = df_old.withColumn("update_date", current_timestamp())
df_old.display()

CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,Customer_rowguid,Customer_ModifiedDate,Person_BusinessEntityID,PersonType,NameStyle,Title,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,Person_rowguid,Person_ModifiedDate,FullName,DimCustomerKey,create_date,update_date


In [0]:
df_new = df_new.drop('old_DimCustomerKey', 'old_CustomerID','old_create_date','old_update_date')

df_new = df_new.withColumn("create_date", current_timestamp())
df_new = df_new.withColumn("update_date", current_timestamp())
df_new.display()

In [0]:
df_new.display()

**Adding dimcustomerkey to the df_new from 1**

In [0]:
df_new = df_new.withColumn("DimCustomerKey", monotonically_increasing_id()+lit(1))

In [0]:
if init_load_flag == 1:
    max_surrogate_key = 0
else:
    max_surr = spark.sql("select max(DimCustomerKey) as max_surrogate_key from advenworks_cata.gold.DimCustomers")
    max_surrogate_key = max_surr.collect()[0]['max_surrogate_key']

In [0]:
df_new = df_new.withColumn("DimCustomerKey", df_new["DimCustomerKey"]+max_surrogate_key)

**Union of df_new and df_old**

In [0]:
df_final = df_new.unionByName(df_old)


In [0]:
df_final.display()

**SCD TYPE 1**

In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists("advenworks_cata.gold.DimCustomers"):
    dlt_obj = DeltaTable.forPath(spark, "abfss://gold@advenworksdl.dfs.core.windows.net/DimCustomers")
    dlt_obj.alias("trg").merge(df_final.alias("src"), "trg.DimCustomerKey == src.DimCustomerKey")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
        
else:
    df_final.write.mode("overwrite")\
        .format("delta")\
        .option("path","abfss://gold@advenworksdl.dfs.core.windows.net/DimCustomers")\
        .saveAsTable("advenworks_cata.gold.DimCustomers")